In [4]:
# install packages
!pip install qdrant_client
!pip install sentence_transformers

In [5]:
# import libraries
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [6]:
# Load dataset and prepare the data
import pandas as pd
df = pd.read_csv(r"C:\Users\aslia\Downloads\top_rated_wines.csv")
df = df[df['variety'].notna()] # remove any NaN values as it blows up serialization
data = df.to_dict('records')
df

,name,region,variety,rating,notes
0,3 Rings Reserve Shiraz 2004,"Barossa Valley, Barossa, South Australia, Aust...",Red Wine,96.0,Vintage Comments : Classic Barossa vintage con...
1,Abreu Vineyards Cappella 2007,"Napa Valley, California",Red Wine,96.0,Cappella is a proprietary blend of two clones ...
2,Abreu Vineyards Cappella 2010,"Napa Valley, California",Red Wine,98.0,Cappella is one of the oldest vineyard sites i...
3,Abreu Vineyards Howell Mountain 2008,"Howell Mountain, Napa Valley, California",Red Wine,96.0,When David purchased this Howell Mountain prop...
4,Abreu Vineyards Howell Mountain 2009,"Howell Mountain, Napa Valley, California",Red Wine,98.0,"As a set of wines, it is hard to surpass the f..."
...,...,...,...,...,...
1360,Lewis Cellars Alec's Blend Red 2002,"Napa Valley, California",Red Wine,96.0,Number 12 on
1361,Lewis Cellars Cabernet Sauvignon 2002,"Napa Valley, California",Red Wine,96.0,Showcasing the unique personalities of small h...
1362,Lewis Cellars Cuvee L Cabernet Sauvignon 2015,"Napa Valley, California",Red Wine,96.0,"Straight from James Fenimore Cooper’s novel, L..."
1363,Lewis Cellars Reserve Cabernet Sauvignon 2010,"Napa Valley, California",Red Wine,96.0,


In [7]:
# Initialize Embedding Model  
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

In [8]:
# Initialize the vector database client Qdrant (in-memory)
qdrant = QdrantClient(":memory:")

In [9]:
# Create a Vector Collection

qdrant.recreate_collection(
    collection_name="top_wines",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),
        distance=models.Distance.COSINE))


C:\Users\aslia\AppData\Local\Temp\ipykernel_32064\2733267425.py:3: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [10]:
# Upload to Qdrant

qdrant.upload_points(
    collection_name="top_wines",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["notes"]).tolist(),
            payload=doc
        ) for idx, doc in enumerate(data) # data is the variable holding all the wines
    ]
)

In [11]:
# Semantic Search

hits = qdrant.search(
    collection_name="top_wines",
    query_vector=encoder.encode("99 points Cabernet Sauvignon from Napa Valley").tolist(),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'name': 'Kapcsandy Family Winery State Lane Cabernet Sauvignon Grand Vin 2017', 'region': 'Yountville, Napa Valley, California', 'variety': 'Red Wine', 'rating': 96.0, 'notes': '100% Cabernet Sauvignon'} score: 0.7492030054523204
{'name': 'Lewis Cellars Cabernet Sauvignon 2002', 'region': 'Napa Valley, California', 'variety': 'Red Wine', 'rating': 96.0, 'notes': 'Showcasing the unique personalities of small hillside vineyards from Pritchard Hill, Oakville and Rutherford, the 2002 Napa Valley Cabernet delivers compelling aromas of mocha, ripe berries, tobacco and sweet oak spice. The wine is 100% Cabernet Sauvignon, complex, rich and focused. With a deep core of black fruit and traces of briar and vanilla, it turns chocolaty and long on the palate with serious, integrated tannins.'} score: 0.7331375680355914
{'name': 'Anakota Helena Montana Vineyard Cabernet Sauvignon 2013', 'region': 'Knights Valley, Sonoma County, California', 'variety': 'Red Wine', 'rating': 96.0, 'notes': 'Blend: 1

C:\Users\aslia\AppData\Local\Temp\ipykernel_32064\3941887777.py:3: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = qdrant.search(


In [12]:
# define a variable to hold the search results

search_results = [hit.payload for hit in hits]

In [15]:
!pip install openai

In [17]:
# Connect Python to Llamafile

from openai import OpenAI

client = OpenAI(
    base_url="http://127.0.0.1:8080/v1", # "http://<Your api-server IP>:port"
    api_key = "sk-no-key-required"
)
completion = client.chat.completions.create(
    model="LLaMA_CPP",
    messages=[
        {"role": "system", "content": "You are chatbot, a wine specialist. Your top priority is to help guide users into selecting amazing wine and guide them with their requests."},
        {"role": "user", "content": "Suggest me an amazing Cabernet Sauvignon from Napa Valley"},
        {"role": "assistant", "content": str(search_results)}
    ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content='I suggest you the Kapcsandy Family Winery State Lane Cabernet Sauvignon Grand Vin 2017 from Yountville, Napa Valley, California. It has a rating of 96.0 and notes of 100% Cabernet Sauvignon.\n', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None)
